In [1]:
# Order volume forecast
import numpy as np  
import pandas as pd  
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn.model_selection import GridSearchCV  
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
data = pd.read_table('../../data/products_sales.txt', delimiter=',')

In [3]:
data.head()

,limit_infor,campaign_type,campaign_level,product_level,resource_amount,email_rate,price,discount_rate,hour_resouces,campaign_fee,orders
0,0,6,0,1,1,0.08,140.0,0.83,93,888,1981
1,0,0,0,1,1,0.10,144.0,0.75,150,836,986
2,0,1,1,1,1,0.12,149.0,0.84,86,1330,1416
3,0,3,1,2,1,0.12,141.0,0.82,95,2273,2368
4,0,0,0,1,1,0.10,146.0,0.59,73,1456,1529


In [4]:
data.dtypes

limit_infor          int64
campaign_type        int64
campaign_level       int64
product_level        int64
resource_amount      int64
email_rate         float64
price              float64
discount_rate      float64
hour_resouces        int64
campaign_fee         int64
orders               int64
dtype: object

In [5]:
data.isnull().sum()

limit_infor        0
campaign_type      0
campaign_level     0
product_level      0
resource_amount    0
email_rate         0
price              2
discount_rate      0
hour_resouces      0
campaign_fee       0
orders             0
dtype: int64

In [6]:
data.describe()

,limit_infor,campaign_type,campaign_level,product_level,resource_amount,email_rate,price,discount_rate,hour_resouces,campaign_fee,orders
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,729.000000,731.000000,731.000000,731.000000,731.000000
mean,0.042408,2.997264,0.683995,1.395349,4.952120,0.474337,162.812071,0.809617,848.176471,3696.391245,4531.079343
std,0.404911,2.004787,0.465233,0.544894,1.838449,0.162913,14.267136,0.077679,686.622488,1908.643139,1932.532346
min,0.000000,0.000000,0.000000,1.000000,1.000000,0.080000,100.000000,0.490000,2.000000,20.000000,22.000000
25%,0.000000,1.000000,0.000000,1.000000,3.000000,0.340000,152.000000,0.770000,315.500000,2497.000000,3199.000000
50%,0.000000,3.000000,1.000000,1.000000,5.000000,0.490000,163.000000,0.820000,713.000000,3662.000000,4563.000000
75%,0.000000,5.000000,1.000000,2.000000,7.000000,0.610000,173.000000,0.870000,1096.000000,4795.500000,6011.500000
max,10.000000,6.000000,1.000000,3.000000,9.000000,0.840000,197.000000,0.980000,3410.000000,33380.000000,8714.000000


In [7]:
col_names = ['limit_infor', 'campaign_type', 'campaign_level', 'product_level']  
for col_name in col_names: 
    unque_value = np.sort(data[col_name].unique()) 
    print ('{:*^50}'.format('{1} unique values:{0}').format(unque_value, col_name)) 

**************limit_infor unique values:[ 0  1 10]***************
**************campaign_type unique values:[0 1 2 3 4 5 6]***************
**************campaign_level unique values:[0 1]***************
**************product_level unique values:[1 2 3]***************


In [8]:
data[data["limit_infor"]==10]

,limit_infor,campaign_type,campaign_level,product_level,resource_amount,email_rate,price,discount_rate,hour_resouces,campaign_fee,orders
705,10,4,1,1,8,0.74,158.0,0.82,606,3784,4390


In [10]:
# process null value and outlier
sales_data = data.fillna(data['price'].mean())  
sales_data = sales_data[sales_data['limit_infor'].isin((0, 1))] 
sales_data['campaign_fee'] = sales_data['campaign_fee'].replace(33380, sales_data['campaign_fee'].mean())   

In [11]:
X = sales_data.ix[:, :-1]  
y = sales_data.ix[:, -1] 

model_gbr = GradientBoostingRegressor() 
parameters = {'loss': ['ls', 'lad', 'huber', 'quantile'],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'alpha': [0.1, 0.3, 0.6, 0.9]}
model_gs = GridSearchCV(estimator=model_gbr, param_grid=parameters, cv=5) 
model_gs.fit(X, y) 
print ('Best score is:', model_gs.best_score_) 
print ('Best parameter is:', model_gs.best_params_)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


Best score is: 0.9316493410140942
Best parameter is: {'alpha': 0.9, 'min_samples_leaf': 3, 'loss': 'huber'}
